<a href="https://colab.research.google.com/github/aparnasree2020/sampledata_casws/blob/main/Run_datacleaning_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

In [ ]:
# Comment after installation
#install.packages('gbm')
#install.packages('Hmisc')


# Load Libraries

In [ ]:
library(tidyverse)
library(gbm)

#library(dplyr)
#require(xgboost)
#library(measures)

# Load CSV datafile

In [ ]:
csv_filename = 'gbm_data_rpm_2023_100K_v2.csv'
df <- read.csv(file =csv_filename, header = TRUE )



# Preprocessing data 

*   Dropping variables that could cause data leakage
*   Dropping variables that connects records to states to not expand the design matrix
*   Dropping variables that has no change



 

In [ ]:
drop <- c("X","limit_uim","incurred_pd_cap","incurred_pd_raw","ee_bi","ee_med","ee_pip","ee_um","ee_uim","ee_comp","ee_tot","claim_count_bi","claim_count_med","claim_count_pip","claim_count_um","claim_count_uim","claim_count_coll","claim_count_comp","multi_policy_discount")
drop_flags <- c("OH_flag","IL_flag","VA_flag","GA_flag","WI_flag","IN_flag","MI_flag","TN_flag","KY_flag","PA_flag","SC_flag","MN_flag","IA_flag","NH_flag","AZ_flag","TX_flag","RI_flag","ME_flag","CT_flag","VT_flag","veh_use_personal",
"veh_use_commute",
"veh_use_farm",
"veh_use_business",
"driver_male",
"driver_married",
"branded_title_loss",
"severe_accident",
"total_loss",
"severe_frame_damage",
"airbag_deployed",
"salvage_auction",
"other_severe_problem",
"failed_emissions",
"failed_safety",
"nonsevere_accident",
"damage_disclosure",
"collision_repair",
"potential_frame_damage",
"rollback",
"severe_problem",
"branded_title",
"potential_damage",
"current_ownership_corp_govt",
"py_2015",
"py_2016",
"py_2017",
"py_2018")
gbm_data_states = df[,!(names(df) %in% drop)]
gbm_data = gbm_data_states[,!(names(gbm_data_states) %in% drop_flags)]

# find columns with character values
char_cols <- sapply(gbm_data, is.character)
# convert "true" and "false" to 1 and 0 in character columns
gbm_data[char_cols] <- lapply(gbm_data[char_cols], function(x) ifelse(x == "true", 1, 0))
gbm_data$claim_count_pd <- ifelse(gbm_data$claim_count_pd > 0.5, 1, 0)

In [ ]:
#write.csv(gbm_data, "Cleaned_data.csv", row.names=FALSE)

## Split data into train and test at 80/20 split

In [ ]:
size <- floor(0.80 * nrow(gbm_data))
set.seed(42)
train_idx <- sample(seq_len(nrow(gbm_data)), size = size)
train <- gbm_data[train_idx,]
test <- gbm_data[-train_idx,]

x_train <- train[, !names(train) %in% "claim_count_pd"]
y_train <- train$claim_count_pd
x_test <- test[, !names(test) %in% "claim_count_pd"]
y_test <- test$claim_count_pd

In [ ]:
write.csv(train, "train_clean.csv", row.names=FALSE)
write.csv(train, "test_clean.csv", row.names=FALSE)

# Fit GBM model and predict output for test data

In [ ]:
# Fit a Gradient Boosting Machine
fit <- gbm(y_train ~ ., data = x_train, distribution = "gaussian", n.trees = 100, interaction.depth = 2)

pred_y_test = predict.gbm(fit, x_test)
pred_y_test_bin = ifelse(pred_y_test > 0.5, 1, 0)


Warning message in gbm.fit(x = x, y = y, offset = offset, distribution = distribution, :
“variable 32: veh_use_personal has no variation.”
Warning message in gbm.fit(x = x, y = y, offset = offset, distribution = distribution, :
“variable 33: veh_use_commute has no variation.”
Warning message in gbm.fit(x = x, y = y, offset = offset, distribution = distribution, :
“variable 34: veh_use_farm has no variation.”
Warning message in gbm.fit(x = x, y = y, offset = offset, distribution = distribution, :
“variable 35: veh_use_business has no variation.”
Warning message in gbm.fit(x = x, y = y, offset = offset, distribution = distribution, :
“variable 36: driver_male has no variation.”
Warning message in gbm.fit(x = x, y = y, offset = offset, distribution = distribution, :
“variable 37: driver_married has no variation.”
Warning message in gbm.fit(x = x, y = y, offset = offset, distribution = distribution, :
“variable 38: branded_title_loss has no variation.”
Warning message in gbm.fit(x = x, y =

In [ ]:
dataout <- data.frame(pred_y_test_bin, y_test)
write.csv(dataout, "GBM_initial.csv", row.names=FALSE)

# End

Unused code below

In [ ]:
#library(Hmisc)
#hist.data.frame(gbm_data$claim_count_pd)